In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.2 MB/s eta 0:00:00


#导入依赖

In [21]:
!pip install onnxruntime

In [22]:
import onnxruntime as ort
import numpy as np
import cv2
from collections import deque

#模型加载

In [29]:
#!wget https://ftrg.zbox.filez.com/v2/delivery/data/95f00b0fc900458ba134f8b180b3f7a1/examples/yolov8_pose/yolov8n-pose.onnx
session = ort.InferenceSession("/content/yolov8n-pose.onnx")
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

#图像预处理函数

In [31]:
def letterbox_resize(image, size, bg_color=114):
    target_width, target_height = size
    image_height, image_width = image.shape[:2]
    scale = min(target_width / image_width, target_height / image_height)
    new_size = (int(image_width * scale), int(image_height * scale))
    resized = cv2.resize(image, new_size)
    canvas = np.full((target_height, target_width, 3), bg_color, dtype=np.uint8)
    offset_x = (target_width - new_size[0]) // 2
    offset_y = (target_height - new_size[1]) // 2
    canvas[offset_y:offset_y + new_size[1], offset_x:offset_x + new_size[0]] = resized
    return canvas, scale, offset_x, offset_y

#姿态分类函数

In [32]:
def classify_pose_state(keypoints_history, move_thresh=8, fall_angle_thresh=30):
    if len(keypoints_history) < 2:
        return "静止"

    prev = keypoints_history[-2][:, :2]
    curr = keypoints_history[-1][:, :2]
    movement = np.linalg.norm(curr - prev, axis=1).mean()

    try:
        shoulder = curr[5], curr[6]
        hip = curr[11], curr[12]
    except IndexError:
        return "静止"

    def angle(p1, p2):
        dx, dy = p2[0] - p1[0], p2[1] - p1[1]
        return np.degrees(np.arctan2(dy, dx))

    torso_angle = angle(np.mean(shoulder, axis=0), np.mean(hip, axis=0))

    if abs(torso_angle) < fall_angle_thresh:
        return "摔倒"
    elif movement > move_thresh:
        return "行走"
    else:
        return "静止"


#单张图片测试



In [33]:
def run_pose_onnx(image_path, onnx_path):
    # 加载模型
    session = ort.InferenceSession(onnx_path)
    input_name = session.get_inputs()[0].name

    # 图像预处理
    img_raw = cv2.imread(image_path)
    img, scale, ox, oy = letterbox_resize(img_raw, (640, 640))
    img_input = img[..., ::-1].astype(np.float32) / 255.0  # BGR to RGB
    img_input = np.transpose(img_input, (2, 0, 1))  # HWC to CHW
    img_input = np.expand_dims(img_input, axis=0)

    # ONNX 推理
    outputs = session.run(None, {input_name: img_input})
    preds = outputs[0]  # shape: [1, N, 56]

    keypoints_history = deque(maxlen=5)

    for det in preds[0]:
        conf = det[4]
        cls = det[5]
        if conf < 0.3 or int(cls) != 0:  # 只处理person
            continue

        # 关键点提取 [17×3]
        kpts = det[6:].reshape(17, 3)  # [x, y, conf]
        kpts[:, 0] = (kpts[:, 0] - ox) / scale
        kpts[:, 1] = (kpts[:, 1] - oy) / scale

        keypoints_history.append(kpts)
        if len(keypoints_history) < 2:
            state = "静止"
        else:
            state = classify_pose_state(keypoints_history)

        # 可视化
        for x, y, c in kpts:
            if c > 0.3:
                cv2.circle(img_raw, (int(x), int(y)), 4, (0, 255, 0), -1)

        cv2.putText(img_raw, f"状态: {state}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("ONNX Pose Detection", img_raw)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [34]:
run_pose_onnx("/content/drive/MyDrive/yolo-pose/people(1602).jpg", "/content/yolov8n-pose.onnx")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

NameError: name 'image' is not defined